In [1]:
using JLD2
using Statistics
using Revise
using Plots 
using Laplacians

In [2]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

nanErr! (generic function with 1 method)

In [3]:
@load "uni_bndry_chimera_all.jl.split02merge02.n10000.h1.0.jld2"
dic1 = copy(dic)
@load "uni_bndry_chimera_all.jl.split02merge02.n100000.h2.0.jld2"
dic2 = copy(dic)
@load "uni_bndry_chimera_all.jl.split02merge02.n1000000.h4.0.jld2"
dic3 = copy(dic)
@load "uni_bndry_chimera_all.jl.split02merge02.n10000000.h8.0.jld2"
dic4 = copy(dic)

Dict{Any,Any} with 84 entries:
  "lamg_build"        => [Inf, Inf, Inf, 26.7645, 513.611, 335.11, Inf, Inf, In…
  "acM2C2_err"        => [2.45124e-9, 4.06815e-9, 4.94714e-9, 4.08455e-9, 2.320…
  "hypre_solve"       => [96.17, 101.79, Inf, 82.77, 224.14, 88.3, 149.98, 102.…
  "acM2C2_solve"      => [87.0844, 116.931, 85.7109, 104.438, 269.125, 145.586,…
  "icc2_tot"          => [98.7038, 107.56, 51.291, 108.274, 102.187, 215.613, 1…
  "acM2C2_tot"        => [166.092, 272.207, 155.58, 208.63, 617.882, 344.836, 5…
  "ac_lst_build"      => [81.096, 150.887, 68.8425, 106.692, 229.62, 175.711, 1…
  "petsc_hypre_build" => [Inf, 961.031, Inf, 745.525, Inf, 395.394, Inf, Inf, 2…
  "lamg_its"          => [Inf, Inf, Inf, 2.0, 7.0, 8.0, Inf, Inf, Inf, 9.0, Inf…
  "lamg_solve"        => [Inf, Inf, Inf, 0.131158, 441.227, 271.115, Inf, Inf, …
  "hypre_its"         => [10.0, 9.0, Inf, 9.0, 11.0, 9.0, 9.0, 9.0, 11.0, 8.0, …
  "cmg_solve"         => [104.152, 153.832, 122.265, 99.5135, 419.037, 207.53,

In [4]:
nanErr!(dic1);
nanErr!(dic2);
nanErr!(dic3);
nanErr!(dic4);

In [5]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)

ffmicro (generic function with 1 method)

In [6]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [11]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(ffK(d["nv"][1]), "&", length(d["testName"]), "&", join(allCells, "&"), "\\\\"))
end

function printAllLines(dics, solvers)
    for d in dics
        printOneLine(d, solvers)
    end
end

printAllLines (generic function with 1 method)

In [12]:
dics = [dic1, dic2, dic3, dic4];

In [13]:
solvers=["ac", "ac-s2m2", "cmg2"]
printAllLines(dics, solvers)

9.54&103&0.379&0.391&0.42&0.543&0.6&0.739&8.23&11.1&17.7\\
97.9&116&0.585&0.684&1.5&0.878&1.19&1.9&2.02&2.83&Inf\\
990&29&1.43&1.62&1.8&1.58&2.01&2.83&1.86&2.52&Inf\\
9950&12&2.7&2.95&3.4&2.84&3.47&5.44&3.58&4.85&7.26\\


In [14]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dics, solvers)

9.54&103&0.904&1.09&2.18&3.56&7.71&21.4&4.38&5.38&9.39\\
97.9&116&1.25&1.75&4.89&11.2&35&167&1.06&1.36&3.87\\
990&29&2.8&4.14&11.4&32.7&86.5&Inf&1.07&1.52&Inf\\
9950&12&5.92&7.73&Inf&Inf&Inf&Inf&1.64&1.83&4.63\\
